### Selected Word Group In Youtube Sentence

In [14]:
import os
import re
import pandas as pd
import numpy as np
import glob
from functools import reduce
from pathlib import Path
import shutil
from itertools import islice

In [15]:
# language pair
lang_folder = "Turkish"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> target language for learner
#lang_pair = "Arabic"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> native language

# native word select for Part 2 
word_start = 0  # 0  # native word start index
word_end = 200 # 28  # native word end index

# source path
source_path = "/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Turkish/Deployment2/Result/1-Twogram With Different Word List Pair/Turkish"

# parameter
file = "Turkish_200_Word_Twogram_With_28_And_172_Word_Pair_Selected"  # without xlsx extention
sheets = "Sheet1"  # 2 gram target, 2 gram hybrid
file_ext = "Twogram_With_28_And_172_Word_Pair_Selected"
first_column = 0  # start column location number
end_column = 1
time_shift = 0.6  # 0.6
sample_num = 10  # 6

# youtube sentence condition
sent_time_start = False  # True, False for activating start time condition
sent_start_time = 700  # sn
random_sample = True  # True, False

In [16]:
path = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/\
Deployment2/Result/7-Selected Word Group In Youtube Sentence/{lang_folder.capitalize()}"

Path(path).mkdir(parents=True, exist_ok=True)

In [17]:
def word_group_youtube(df, search_list, target_column, sample_num):
    '''
    word_group_youtube(df_youtube_sentence, search_list, "sentence", 6)
    ''' 
    df_search_result = pd.DataFrame()
    for j in search_list:
        try:
            df_select = df[df[f"{target_column}"].str.contains(fr"(?:\s|^){j}(?:\s|$)", na=True)].sample(sample_num)
        except:
            df_select = df[df[f"{target_column}"].str.contains(fr"(?:\s|^){j}(?:\s|$)", na=True)].head(sample_num)
        #df_result = df[df[f"{target_column}"].str.contains(fr"(?:\s|^){j}(?:\s|$)", na=True)]  # sentence length part
        #df_result.sort_values(f"{target_column}",key=lambda x:x.str.len(), inplace=True)
        #df_select = df_result.head(sample_num)
        df_select.insert(0,"search_string",j)
        df_search_result = pd.concat([df_search_result,df_select], axis=0)
    df_search_result.reset_index(inplace=True, drop=True)
    
    return df_search_result

In [18]:
def word_group_time_loc(df, search, start_sent, end_sent, sent, sent_video_id):
    '''
    word_group_time_loc(df_search_result, "search_string", "start_time", "end_time", "sentence", "video_id")
    '''
    df.reset_index(drop=True, inplace=True)
    word_time_loc_list = []
    for i in range(len(df)):
        word = df.loc[i,f"{search}"]
        start_time = df.loc[i,f"{start_sent}"]
        end_time = df.loc[i,f"{end_sent}"]
        sentence = df.loc[i,f"{sent}"]
        video_id = df.loc[i,f"{sent_video_id}"]
        time_length = end_time-start_time
        sentence_length = len(sentence)
        time_length_ratio = time_length/sentence_length
        loc_list = []
        for j in re.finditer(fr"(?:\s|^){word}(?:\s|$)", sentence, re.IGNORECASE|re.UNICODE):
            loc_list.append(j)
            start = loc_list[0].start()
            end = loc_list[0].end()
            start_loc = start_time+(start*time_length_ratio)
            end_loc = start_time+(end*time_length_ratio)
        word_time_loc_list.append([word,start_loc,end_loc,sentence,video_id])
    df_word_time_loc = pd.DataFrame(word_time_loc_list, columns=[f"{search}",f"{start_sent}",f"{end_sent}",f"{sent}",f"{sent_video_id}"])

    return df_word_time_loc

In [19]:
def word_count_result(df, column_list, set_condition=False): # df is dataframe, column_list is list value
    '''
    word_count_bool(df, column_list): df columns word count for word frequency\n
    df is dataframe, column_list is list value\n
    word_count_bool(df, ["word","twogram"]):
    '''
    list_all = []
    for i in df.loc[:,[x for x in column_list]].columns:
        if set_condition:
            var_list = set(df[f"{i}"].dropna().tolist())
        else:
            var_list = df[f"{i}"].dropna().tolist()
        for j in var_list:
            list_all.append(j)
    text = " ".join(list_all)
    word_list = re.findall(r"\w+",text, re.UNICODE)
    df_word_list = pd.DataFrame(word_list, columns=["word"])
    #df_word_list.rename(columns={0:"word"}, inplace=True)
    df_word_count = pd.DataFrame(df_word_list.value_counts())
    df_word_count.reset_index(inplace=True)
    df_word_count.rename(columns={0:"word_count"}, inplace=True)
    df_word_count.sort_values("word_count", ascending=False, inplace=True)
    df_word_count.reset_index(inplace=True, drop=True)
    
    return  df_word_count

In [20]:
df_word_all = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Deployment/Data/Word/Word_Merge_Preprocess.xlsx")
df_word_all

,word,frequency
0,bir,18835735
1,bu,11062659
2,ne,8025880
3,ve,7766036
4,için,5484109
...,...,...
987924,karneleme,5
987925,karnaya,5
987926,dörtlümüzün,5
987927,karnavalınız,5


In [21]:
df_word_select = df_word_all.iloc[word_start:word_end,]
df_word_select

,word,frequency
0,bir,18835735
1,bu,11062659
2,ne,8025880
3,ve,7766036
4,için,5484109
...,...,...
195,saat,399989
196,onunla,399330
197,yapıyorsun,398274
198,neler,397377


In [22]:
df_youtube_sentence = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Youtube/Result/{lang_folder.capitalize()}/Sentence Clean Merge/Clean_Youtube_Sentence_Merge_Result.csv")
df_youtube_sentence

,start_time,end_time,sentence,video_id
0,00:00:00.294,00:00:03.294,bu dizinin betimlemesi staff fi lm tarafından,fI1BI4d5KFU
1,00:00:03.375,00:00:06.375,sesli betimleme derneğine yaptırılmıştır,fI1BI4d5KFU
2,00:00:06.462,00:00:09.462,wwwsebederorg,fI1BI4d5KFU
3,00:00:10.267,00:00:11.394,mumu,fI1BI4d5KFU
4,00:00:11.835,00:00:14.061,bak şu an sinir katsayım hat safhada,fI1BI4d5KFU
...,...,...,...,...
3934202,00:10:00.240,00:10:07.818,videoyu bitirmeden şunu da söylemek isterim ki...,dRssR_apbR0
3934203,00:10:07.994,00:10:13.534,bu nedenle ulaşımda insanların yarısına yakını...,dRssR_apbR0
3934204,00:10:14.745,00:10:17.924,danimarkayla alakalı şimdilik söyleyeceklerim ...,dRssR_apbR0
3934205,00:10:18.047,00:10:22.398,video hoşunuza gittiyse beğenmeyi ve kanalıma ...,dRssR_apbR0


In [23]:
df_youtube_sentence['start_time'] = pd.to_timedelta(df_youtube_sentence['start_time']) # data type converted timedelta for second 
df_youtube_sentence['end_time'] = pd.to_timedelta(df_youtube_sentence['end_time'])

In [24]:
df_youtube_sentence['start_time'] = df_youtube_sentence['start_time'].apply(lambda x: x.total_seconds()) # convert seconds
df_youtube_sentence['end_time'] = df_youtube_sentence['end_time'].apply(lambda x: x.total_seconds())
df_youtube_sentence

,start_time,end_time,sentence,video_id
0,0.294,3.294,bu dizinin betimlemesi staff fi lm tarafından,fI1BI4d5KFU
1,3.375,6.375,sesli betimleme derneğine yaptırılmıştır,fI1BI4d5KFU
2,6.462,9.462,wwwsebederorg,fI1BI4d5KFU
3,10.267,11.394,mumu,fI1BI4d5KFU
4,11.835,14.061,bak şu an sinir katsayım hat safhada,fI1BI4d5KFU
...,...,...,...,...
3934202,600.240,607.818,videoyu bitirmeden şunu da söylemek isterim ki...,dRssR_apbR0
3934203,607.994,613.534,bu nedenle ulaşımda insanların yarısına yakını...,dRssR_apbR0
3934204,614.745,617.924,danimarkayla alakalı şimdilik söyleyeceklerim ...,dRssR_apbR0
3934205,618.047,622.398,video hoşunuza gittiyse beğenmeyi ve kanalıma ...,dRssR_apbR0


In [25]:
if sent_time_start:
    df_youtube_select = df_youtube_sentence[df_youtube_sentence["start_time"] <= sent_start_time]
    df_youtube_select.reset_index(drop=True, inplace=True)
else:
    df_youtube_select = df_youtube_sentence

if random_sample:
    df_youtube_select = df_youtube_select.sample(len(df_youtube_select))
    df_youtube_select.reset_index(drop=True, inplace=True)
else:
    pass

df_youtube_select

,start_time,end_time,sentence,video_id
0,9232.432,9237.398,ben de evde kendim yaptım hem de hiç masraf fa...,JrJs2vfea4g
1,189.340,192.981,sence bizim kızımız mı olacak öyle mi hissediy...,aL20HP_5vdQ
2,1887.257,1891.170,nazlı kız duydun mu komiser rıza var ya,8_VkxmHeMAk
3,150.610,153.046,hoş geldin tarık hoş buldum,z_0AVXYQIgw
4,732.326,734.412,ah şükrü ah,5k5s_IiQzXc
...,...,...,...,...
3934202,3145.399,3147.304,i ki saat önce burada olması lazımdı,3ZFfL3wl1so
3934203,173.680,175.480,bir sürü şive de öğrendiniz o zaman,O3iT4_8jUVw
3934204,70.960,75.730,ediyordu şimdi biz bir açalım neden ısınıyor b...,OCbeS5MVDXg
3934205,2952.856,2955.454,ben de seni özlediğim için ona takıldım geldim,twGX5mN6Mcs


In [26]:
df_word_group_select = pd.read_excel(f"{source_path}/{file}.xlsx", sheet_name= f"{sheets}")
df_word_group_select

,twogram,frequency
0,ne yapıyorsun,236176
1,hiçbir şey,209701
2,ne demek,188846
3,gerçekten mi,141653
4,ne zaman,131571
...,...,...
103,beni burada,6045
104,kadar önemli,5913
105,seni hiç,5715
106,gibi geliyor,5188


In [ ]:
# Option 
# Skip if it doesn't need
df_remove = pd.read_excel(f"")
df_remove

In [ ]:
remove_videoid_list = list(set(df_remove["video_id"]))
#remove_videoid_list

In [ ]:
len(remove_videoid_list)

In [ ]:
# Option
df_youtube_select = df_youtube_select[~df_youtube_select["video_id"].isin(remove_videoid_list)]
df_youtube_select

In [27]:
search_list = []
for i in range(len(df_word_group_select)):
    for j in df_word_group_select.columns[first_column:end_column]:
        string = df_word_group_select.loc[i,j]
        
        if pd.isnull(string) == False and string != 'nan':
            search_list.append(string.strip())
            search_list = list(set(search_list))  # for unique value
        else:
            pass

In [28]:
len(search_list)

108

In [29]:
df_search_result = word_group_youtube(df_youtube_select, search_list, "sentence", sample_num)
df_search_result

,search_string,start_time,end_time,sentence,video_id
0,mu var,4716.054,4717.764,doktorluk mu var sende,v8eY_Xa5SGw
1,mu var,6043.780,6046.280,oğlum sıkıntılı bir durum mu var semanın yanın...,OImI_S8C2vo
2,mu var,1461.020,1462.240,bir sorun mu var,9k02Yj8lIKM
3,mu var,221.708,226.327,çıkınca ben bir sorun mu var diye merak ettim,j55dDQ7LgG0
4,mu var,1057.426,1059.447,niye celal çocuk mu var senin karşında,im2J3GRCnGk
...,...,...,...,...,...
1063,hayır efendim,981.549,984.682,hayır efendim öyle bir yüzük takmakla damat fa...,x6aEnqpTUBo
1064,hayır efendim,8963.980,8964.837,hayır efendim,LFHDU8S0yFg
1065,hayır efendim,1820.402,1823.132,hayır efendim sen itır sırık ve o bacaksız anası,_l6NM6NEKdI
1066,hayır efendim,4898.040,4901.440,hayır efendim hayır suzan tartışma benimle,oYykJu6A7Dg


In [30]:
df_word_group_time_loc_result = word_group_time_loc(df_search_result, "search_string", "start_time", "end_time", "sentence", "video_id")
df_word_group_time_loc_result

,search_string,start_time,end_time,sentence,video_id
0,mu var,4716.753545,4717.375364,doktorluk mu var sende,v8eY_Xa5SGw
1,mu var,6044.896071,6045.253214,oğlum sıkıntılı bir durum mu var semanın yanın...,OImI_S8C2vo
2,mu var,1461.706250,1462.240000,bir sorun mu var,9k02Yj8lIKM
3,mu var,223.863533,224.684689,çıkınca ben bir sorun mu var diye merak ettim,j55dDQ7LgG0
4,mu var,1058.276947,1058.702421,niye celal çocuk mu var senin karşında,im2J3GRCnGk
...,...,...,...,...,...
1063,hayır efendim,981.549000,982.292424,hayır efendim öyle bir yüzük takmakla damat fa...,x6aEnqpTUBo
1064,hayır efendim,8963.980000,8964.837000,hayır efendim,LFHDU8S0yFg
1065,hayır efendim,1820.402000,1821.198250,hayır efendim sen itır sırık ve o bacaksız anası,_l6NM6NEKdI
1066,hayır efendim,4898.040000,4899.173333,hayır efendim hayır suzan tartışma benimle,oYykJu6A7Dg


In [31]:
df_word_group_time_loc_result.start_time = df_word_group_time_loc_result.start_time.apply(lambda x: (x-time_shift))
df_word_group_time_loc_result.end_time = df_word_group_time_loc_result.end_time.apply(lambda x: (x+time_shift))
df_word_group_time_loc_result

,search_string,start_time,end_time,sentence,video_id
0,mu var,4716.153545,4717.975364,doktorluk mu var sende,v8eY_Xa5SGw
1,mu var,6044.296071,6045.853214,oğlum sıkıntılı bir durum mu var semanın yanın...,OImI_S8C2vo
2,mu var,1461.106250,1462.840000,bir sorun mu var,9k02Yj8lIKM
3,mu var,223.263533,225.284689,çıkınca ben bir sorun mu var diye merak ettim,j55dDQ7LgG0
4,mu var,1057.676947,1059.302421,niye celal çocuk mu var senin karşında,im2J3GRCnGk
...,...,...,...,...,...
1063,hayır efendim,980.949000,982.892424,hayır efendim öyle bir yüzük takmakla damat fa...,x6aEnqpTUBo
1064,hayır efendim,8963.380000,8965.437000,hayır efendim,LFHDU8S0yFg
1065,hayır efendim,1819.802000,1821.798250,hayır efendim sen itır sırık ve o bacaksız anası,_l6NM6NEKdI
1066,hayır efendim,4897.440000,4899.773333,hayır efendim hayır suzan tartışma benimle,oYykJu6A7Dg


In [32]:
df_word_group_time_loc_result.start_time = df_word_group_time_loc_result.start_time.apply(lambda x: round(x))
df_word_group_time_loc_result.end_time = df_word_group_time_loc_result.end_time.apply(lambda x: round(x))
df_word_group_time_loc_result 

,search_string,start_time,end_time,sentence,video_id
0,mu var,4716,4718,doktorluk mu var sende,v8eY_Xa5SGw
1,mu var,6044,6046,oğlum sıkıntılı bir durum mu var semanın yanın...,OImI_S8C2vo
2,mu var,1461,1463,bir sorun mu var,9k02Yj8lIKM
3,mu var,223,225,çıkınca ben bir sorun mu var diye merak ettim,j55dDQ7LgG0
4,mu var,1058,1059,niye celal çocuk mu var senin karşında,im2J3GRCnGk
...,...,...,...,...,...
1063,hayır efendim,981,983,hayır efendim öyle bir yüzük takmakla damat fa...,x6aEnqpTUBo
1064,hayır efendim,8963,8965,hayır efendim,LFHDU8S0yFg
1065,hayır efendim,1820,1822,hayır efendim sen itır sırık ve o bacaksız anası,_l6NM6NEKdI
1066,hayır efendim,4897,4900,hayır efendim hayır suzan tartışma benimle,oYykJu6A7Dg


In [33]:
df_word_group_time_loc_result["video_url"] = "https://www.youtube.com/watch?v="+df_word_group_time_loc_result['video_id'].map(str)+"&t="+df_word_group_time_loc_result['start_time'].map(str)+"s"
df_word_group_time_loc_result

,search_string,start_time,end_time,sentence,video_id,video_url
0,mu var,4716,4718,doktorluk mu var sende,v8eY_Xa5SGw,https://www.youtube.com/watch?v=v8eY_Xa5SGw&t=...
1,mu var,6044,6046,oğlum sıkıntılı bir durum mu var semanın yanın...,OImI_S8C2vo,https://www.youtube.com/watch?v=OImI_S8C2vo&t=...
2,mu var,1461,1463,bir sorun mu var,9k02Yj8lIKM,https://www.youtube.com/watch?v=9k02Yj8lIKM&t=...
3,mu var,223,225,çıkınca ben bir sorun mu var diye merak ettim,j55dDQ7LgG0,https://www.youtube.com/watch?v=j55dDQ7LgG0&t=...
4,mu var,1058,1059,niye celal çocuk mu var senin karşında,im2J3GRCnGk,https://www.youtube.com/watch?v=im2J3GRCnGk&t=...
...,...,...,...,...,...,...
1063,hayır efendim,981,983,hayır efendim öyle bir yüzük takmakla damat fa...,x6aEnqpTUBo,https://www.youtube.com/watch?v=x6aEnqpTUBo&t=...
1064,hayır efendim,8963,8965,hayır efendim,LFHDU8S0yFg,https://www.youtube.com/watch?v=LFHDU8S0yFg&t=...
1065,hayır efendim,1820,1822,hayır efendim sen itır sırık ve o bacaksız anası,_l6NM6NEKdI,https://www.youtube.com/watch?v=_l6NM6NEKdI&t=...
1066,hayır efendim,4897,4900,hayır efendim hayır suzan tartışma benimle,oYykJu6A7Dg,https://www.youtube.com/watch?v=oYykJu6A7Dg&t=...


In [34]:
search_result_num = df_word_group_time_loc_result["search_string"].nunique()
search_result_num

108

In [35]:
search_result_set = set(df_word_group_time_loc_result["search_string"])
search_all_set = set(search_list)

In [36]:
search_all_set.difference(search_result_set)

set()

In [37]:
df_result_word_manuel_count = word_count_result(df_word_group_time_loc_result, df_word_group_time_loc_result.columns[0:1], set_condition=True)
df_result_word_manuel_count

,word,word_count
0,kadar,5
1,hayır,5
2,sen,5
3,beni,5
4,ben,5
...,...,...
89,belki,1
90,başka,1
91,şeyler,1
92,tek,1


In [ ]:
# # Option merge
# df_word_count_merge = pd.merge(df_result_word_manuel_count, df_word_select[["word"]], how="inner", on="word")
# df_word_count_merge.drop_duplicates(inplace=True)
# df_word_count_merge.sort_values(by="word_count", ascending=False, inplace=True)
# df_word_count_merge.reset_index(drop=True, inplace=True)
# df_word_count_merge

In [38]:
set_word = set(df_word_select["word"])
#set_count = set(df_word_count_merge["word"])  # Option merge
set_count = set(df_result_word_manuel_count["word"])

In [39]:
set_word.difference(set_count)

{'adam',
 'anne',
 'asla',
 'aslında',
 'baba',
 'bak',
 'bakalım',
 'bence',
 'benimle',
 'bile',
 'biliyor',
 'bilmiyorum',
 'biri',
 'birkaç',
 'birlikte',
 'bize',
 'bizi',
 'bugün',
 'buna',
 'bütün',
 'büyük',
 'dakika',
 'devam',
 'diye',
 'dostum',
 'dur',
 'ederim',
 'en',
 'et',
 'eğer',
 'gel',
 'geldi',
 'gerek',
 'geri',
 'gerçek',
 'göre',
 'gün',
 'hadi',
 'hakkında',
 'hala',
 'harika',
 'hemen',
 'herkes',
 'hey',
 'iki',
 'ile',
 'ilk',
 'ister',
 'istiyorum',
 'izin',
 'içinde',
 'iş',
 'işte',
 'kendi',
 'kız',
 'lazım',
 'merhaba',
 'musun',
 'mü',
 'mısın',
 'neler',
 'ol',
 'olabilir',
 'olacak',
 'olan',
 'olarak',
 'oldu',
 'olduğunu',
 'olmak',
 'olsun',
 'olur',
 'oluyor',
 'ona',
 'onları',
 'onun',
 'onunla',
 'orada',
 'para',
 'pekala',
 'peki',
 'saat',
 'sanırım',
 'selam',
 'seninle',
 'siz',
 'size',
 'sizi',
 'son',
 'tam',
 'tamam',
 'teşekkür',
 'tüm',
 'vardı',
 'yani',
 'yardım',
 'yeni',
 'yoksa',
 'zaten',
 'çünkü',
 'önce',
 'özür',
 'üç',
 'ş

In [40]:
df_word_group_time_loc_result.to_excel(f"{lang_folder.capitalize()}_{search_result_num}_{file_ext}_\
{sample_num}_Youtube_{time_shift}s_Timeshift_Result.xlsx", index=False) 

In [ ]:
# df_result_word_manuel_count.to_excel(f"{lang_folder.capitalize()}_{search_result_num}_{file_ext}_\
# {sample_num}_Youtube_{time_shift}s_Timeshift_Result_Word_Count.xlsx", index=False) 

#### Copy Move And Delete

In [41]:
output_file = glob.glob(f"{lang_folder.capitalize()}_*_{sample_num}_Youtube_{time_shift}s_Timeshift_Result*.xlsx")
output_file

['Turkish_108_Twogram_With_28_And_172_Word_Pair_Selected_10_Youtube_0.6s_Timeshift_Result.xlsx']

In [42]:
for y in output_file:
    source = y # source directory
    destination = path
    shutil.copy2(source, destination)

In [43]:
for z in output_file:
    try:
        os.remove(z)
    except:
        pass

#### Temp

In [ ]:
set_result = set(df_word_group_time_loc_result["search_string"].unique())
set_selected = set(search_list)

In [ ]:
df_not_in_result = pd.DataFrame(set_selected.difference(set_result), columns=["not_in_youtube_result"])
df_not_in_result

In [ ]:
not_in_result_num = len(df_not_in_result)
not_in_result_num

In [ ]:
#df_not_in_result.to_excel(f"{lang_folder.capitalize()}_{not_in_result_num}_{file_ext}_\
#Not_In_{sample_num}_Youtube_{time_shift}s_Timeshift_Result.xlsx", index=False) 

In [ ]:
search_list2 = df_not_in_result["not_in_youtube_result"].to_list()

In [ ]:
df_search_result2 = word_group_youtube(df_youtube_sentence, search_list2, "sentence", sample_num)
df_search_result2

In [ ]:
search_result2_num = df_search_result2["search_string"].nunique()
search_result2_num

In [ ]:
df_word_group_time_loc_result2 = word_group_time_loc(df_search_result2, "search_string", "start_time", "end_time", "sentence", "video_id")
df_word_group_time_loc_result2

In [ ]:
df_word_group_time_loc_result2.start_time = df_word_group_time_loc_result2.start_time.apply(lambda x: (x-time_shift))
df_word_group_time_loc_result2.end_time = df_word_group_time_loc_result2.end_time.apply(lambda x: (x+time_shift))
df_word_group_time_loc_result2

In [ ]:
df_word_group_time_loc_result2["video_url"] = "https://www.youtube.com/watch?v="+df_word_group_time_loc_result2['video_id'].map(str)+"&t="+df_word_group_time_loc_result2['start_time'].map(str)+"s"
df_word_group_time_loc_result2

In [ ]:
search_result_num2 = df_word_group_time_loc_result2["search_string"].nunique()
search_result_num2

In [ ]:
df_word_group_time_loc_result2.to_excel(f"{lang_folder.capitalize()}_{search_result_num2}_{file_ext}_\
{sample_num}_Youtube_{time_shift}s_Timeshift_Result2.xlsx", index=False) 